In [15]:
import httpx
import pandas as pd
from datetime import datetime

def remove_keys_recursive(d, keys_to_remove):
    if isinstance(d, dict):
        return {
            key: remove_keys_recursive(value, keys_to_remove)
            for key, value in d.items()
            if key not in keys_to_remove
        }
    elif isinstance(d, list):
        return [remove_keys_recursive(item, keys_to_remove) for item in d]
    else:
        return d

def rename_quantity_sold(product):
    if "quantity_sold" in product:
        quantity_sold = product.pop("quantity_sold")
        product["quantity_sold_value"] = quantity_sold["value"]
    return product

def make_api_request(url, params, headers):
    with httpx.Client() as client:
        response = client.get(url, params=params, headers=headers)
    return response

def flatten_data(item):
    flattened_data = {
        key: value for key, value in item.items() if key not in ["visible_impression_info"]
    }
    
    visible_impression_info = item.get("visible_impression_info", {}).get("amplitude", {})

    fields_to_extract = [
        "category_l1_name",
        "category_l2_name",
        "category_l3_name",
        "seller_type",
        "primary_category_name",
        "is_imported",
    ]

    flattened_data.update({field: visible_impression_info.get(field, None) for field in fields_to_extract})

    return flattened_data

def save_to_local(data, timestamp):
    # Remove specified keys from all levels of the nested dictionary
    keys_to_remove = ['impression_info', 'badges_new']
    cleaned_data = remove_keys_recursive(data, keys_to_remove)

    # Rename the "quantity_sold" field to "quantity_sold_value"
    cleaned_data = [rename_quantity_sold(product) for product in cleaned_data]

    # Flatten the nested structure
    cleaned_data = [flatten_data(product) for product in cleaned_data]

    # Convert the cleaned data to a Pandas DataFrame
    pd_df = pd.DataFrame(cleaned_data)

    # Convert timestamp to Unix time
    unix_timestamp = int(datetime.timestamp(timestamp))

    pd_df["ingestion_dt_unix"] = unix_timestamp
    pd_df.rename(columns={"id": "tiki_pid"}, inplace=True)
    pd_df["ingestion_date"] = pd.to_datetime(pd.to_datetime(pd_df["ingestion_dt_unix"], unit='s').dt.date)

    pd_df["quantity_sold_value"] = pd_df["quantity_sold_value"] + 4

    # Specify local path for saving
    local_path_csv = f'./data/raw_116532_{unix_timestamp}.csv'
    local_path_parquet = f'./data/raw_116532_{unix_timestamp}.parquet'
    
    # Write the DataFrame to local CSV file
    pd_df.to_csv(local_path_csv, index=False)

    # Write the DataFrame to local Parquet file
    #pd_df.to_parquet(local_path_parquet, index=False)

    print(f"Local Data saved successfully:\nCSV: {local_path_csv}\nParquet: {local_path_parquet}")

    return pd_df

# API request
url = 'https://api.tiki.vn/seller-store/v2/collections/116532/products'
params = {'limit': 100, 'cursor': 40}
headers = {'x-source': 'local', 'Host': 'api.tiki.vn'}

response = make_api_request(url, params, headers)

if response.status_code == 200:
    data = response.json()['data']

    timestamp = datetime.now()

    # Save to local files
    cleaned_data_out = save_to_local(data, timestamp)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Display the Pandas DataFrame
print(cleaned_data_out.head(2))


Local Data saved successfully:
CSV: ./data/raw_116532_1710308519.csv
Parquet: ./data/raw_116532_1710308519.parquet
    tiki_pid            sku  \
0     548888  1224949455366   
1  273532024  9402014918334   

                                                name  \
0    Combo 4 Hũ Thủy Tinh Quattro Bormioli Rocco QS7   
1  [Combo 3 lon còn 1tr550] Sữa Bột Nutifood Grow...   

                                             url_key  \
0  combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs...   
1  sua-bot-nutifood-growplus-suy-dinh-duong-1-5kg...   

                                            url_path   price  discount  \
0  combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs...  357000    128000   
1  sua-bot-nutifood-growplus-suy-dinh-duong-1-5kg...  565000    319000   

   discount_rate  rating_average  review_count  ...   category_l2_name  \
0             26             4.6            10  ...    Dụng cụ nhà bếp   
1             36             5.0             7  ...  Dinh dưỡng cho bé   

       

In [1]:
from util_minio import MinioHandler

In [2]:
minio_handler = MinioHandler()
#s3.Bucket('tiki').upload_file('./data/raw_116532_1710308519.csv', 'raw_116532_1710308519.csv')

In [4]:
minio_handler.upload_file("tiki", "raw_116532_1710308519.csv", "./data/raw_116532_1710308519.csv")

Exception: Error uploading file: [Errno 2] No such file or directory: 'raw_116532_1710308519.csv'

In [21]:
# Get the object from the bucket
obj = s3.Bucket('tiki').Object('raw_116532_1710308519.csv').get()

# Read the object data as bytes
data = obj['Body'].read()

# Decode the data if necessary (e.g., for text files)
data_str = data.decode("utf-8")  # Replace encoding if needed

# Process the downloaded data
print(data_str)

tiki_pid,sku,name,url_key,url_path,price,discount,discount_rate,rating_average,review_count,thumbnail_url,seller_product_id,original_price,isGiftAvailable,is_from_official_store,tiki_verified,seller_product_sku,origin,availability,primary_category_path,shippable,seller_name,tiki_hero,imported,thumbnail_width,is_nextday_delivery,is_tikinow_delivery,seller_id,is_normal_delivery,brand_name,primary_category_name,brand_id,thumbnail_height,is_authentic,productset_id,master_product_sku,quantity_sold_value,category_l1_name,category_l2_name,category_l3_name,seller_type,is_imported,layout_type,video_url,fastest_delivery_time,order_route,ingestion_dt_unix,ingestion_date
548888,1224949455366,Combo 4 Hũ Thủy Tinh Quattro Bormioli Rocco QS7,combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs7-p548888,combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs7-p548888.html?spid=273195,357000,128000,26,4.6,10,https://salt.tikicdn.com/cache/280x280/media/catalog/product/1/2/1224949455366.u2566.d20161018.t160413.917729.

In [22]:
print(type(data_str))

<class 'str'>


In [26]:
df = pd.read_csv(io.BytesIO(data)) 
df

,tiki_pid,sku,name,url_key,url_path,price,discount,discount_rate,rating_average,review_count,...,category_l2_name,category_l3_name,seller_type,is_imported,layout_type,video_url,fastest_delivery_time,order_route,ingestion_dt_unix,ingestion_date
0,548888,1224949455366,Combo 4 Hũ Thủy Tinh Quattro Bormioli Rocco QS7,combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs...,combo-4-hu-thuy-tinh-quattro-bormioli-rocco-qs...,357000,128000,26,4.6,10,...,Dụng cụ nhà bếp,Dụng cụ chứa đựng thực phẩm,OFFICIAL_STORE,True,NaN,NaN,NaN,NaN,1710308519,2024-03-13
1,273532024,9402014918334,[Combo 3 lon còn 1tr550] Sữa Bột Nutifood Grow...,sua-bot-nutifood-growplus-suy-dinh-duong-1-5kg...,sua-bot-nutifood-growplus-suy-dinh-duong-1-5kg...,565000,319000,36,5.0,7,...,Dinh dưỡng cho bé,NaN,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,1710308519,2024-03-13
2,56094765,9110393150887,Bộ 4 kem đánh răng làm trắng răng Colgate Opti...,bo-4-kem-danh-rang-lam-trang-rang-colgate-opti...,bo-4-kem-danh-rang-lam-trang-rang-colgate-opti...,165000,35000,18,0.0,0,...,Chăm sóc cá nhân,Chăm sóc răng miệng,OFFICIAL_STORE,False,grid_view,https://salt.tikicdn.com/ts/video/2d/b5/e7/e1c...,NaN,NaN,1710308519,2024-03-13
3,184476147,2833967241350,Bộ đôi Sữa rửa mặt tạo bọt Collagen cho da Khô...,bo-doi-sua-rua-mat-tao-bot-collagen-cho-da-kho...,bo-doi-sua-rua-mat-tao-bot-collagen-cho-da-kho...,173000,65000,27,5.0,1,...,Chăm sóc da mặt,NaN,OFFICIAL_STORE,True,NaN,NaN,NaN,NaN,1710308519,2024-03-13
4,272125975,7490306649809,Nước tẩy trang Simple Micellar làm sạch trang ...,nuoc-tay-trang-simple-micellar-lam-sach-trang-...,nuoc-tay-trang-simple-micellar-lam-sach-trang-...,189000,31000,14,5.0,8,...,Chăm sóc da mặt,NaN,OFFICIAL_STORE,True,NaN,NaN,NaN,NaN,1710308519,2024-03-13
5,115669047,7442834927703,Nước lau sàn Gift Tinh chất Thiên Nhiên _ can ...,nuoc-lau-san-gift-tinh-chat-thien-nhien-_-can-...,nuoc-lau-san-gift-tinh-chat-thien-nhien-_-can-...,83400,9500,0,5.0,2,...,Chăm sóc nhà cửa,Vệ sinh nhà bếp,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,1710308519,2024-03-13
6,270851667,8398718337298,Mặt nạ giấy dưỡng da Hàn Quốc innisfree My Rea...,mat-na-giay-duong-da-han-quoc-innisfree-my-rea...,mat-na-giay-duong-da-han-quoc-innisfree-my-rea...,23000,4000,15,5.0,20,...,Chăm sóc da mặt,NaN,OFFICIAL_STORE,True,NaN,NaN,NaN,NaN,1710308519,2024-03-13
7,252608882,4825599512084,Tã/bỉm quần HUGGIES SKINCARE MEGA JUMBO size X...,thung-ta-bim-quan-huggies-skincare-mega-jumbo-...,thung-ta-bim-quan-huggies-skincare-mega-jumbo-...,365000,280750,43,4.8,306,...,"Tã, Bỉm",NaN,OFFICIAL_STORE,False,grid_view,NaN,NaN,NaN,1710308519,2024-03-13
8,252608902,1122504140413,Tã/bỉm quần HUGGIES SKINCARE gói SUPER JUMBO s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,270000,203130,43,5.0,36,...,"Tã, Bỉm",NaN,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,1710308519,2024-03-13
9,252608891,1033225203769,Tã/bỉm quần HUGGIES SKINCARE gói SUPER JUMBO s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,279000,194130,41,5.0,161,...,"Tã, Bỉm",NaN,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,1710308519,2024-03-13
